In [1]:
import pandas as pd
import numpy as np
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import concurrent.futures  # Add this import
from multiprocessing import Pool
from functools import partial
import logging
from multiprocessing import Pool, Lock, current_process
from functools import partial
import sys
import pandas as pd
import subprocess
from pathlib import Path
import os
from multiprocessing import Pool, cpu_count
import numpy as np
from functools import partial
import tempfile


In [2]:
# Set the current working directory
os.chdir('/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_MECP2')

In [3]:
import gzip
import shutil

# Decompress the GTF file
with gzip.open('gencode.vM10.annotation.gtf.gz', 'rb') as f_in:
    with open('gencode.vM10.annotation.gtf', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [4]:
def process_deseq_results(deseq_file, padj_threshold=0.05, lfc_threshold=1):
    """
    Process DESeq2 results using vectorized operations
    """
    # Read the entire file - we'll use vectorized operations for speed
    df = pd.read_csv(deseq_file)
    
    # Use vectorized operations for categorization
    up_mask = (df['padj'] < padj_threshold) & (df['log2FoldChange'] > lfc_threshold)
    down_mask = (df['padj'] < padj_threshold) & (df['log2FoldChange'] < -lfc_threshold)
    non_mask = ~(up_mask | down_mask)
    
    return {
        'up': df.loc[up_mask, 'gene'].tolist(),
        'down': df.loc[down_mask, 'gene'].tolist(),
        'non': df.loc[non_mask, 'gene'].tolist()
    }

def create_bed_file(args):
    """
    Worker function for parallel BED file creation
    """
    genes, gtf_file, output_file, category = args
    
    # Create a temporary file with gene list
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as temp_file:
        temp_file.write('\n'.join(genes))
        temp_file_path = temp_file.name
    
    try:
        # Use grep for faster filtering and awk for processing
        cmd = f"""grep -F -f {temp_file_path} {gtf_file} | 
                 awk -F'\t' '$3=="gene" {{
                     if (match($9, /gene_name "([^"]+)"/, n))
                         print $1"\t"$4-1"\t"$5"\t"n[1]"\t.\t"$7
                 }}' > {output_file}"""
        
        subprocess.run(cmd, shell=True, check=True)
    finally:
        # Clean up temporary file
        os.unlink(temp_file_path)
    
    return output_file, category

def process_tissue_data(args):
    """
    Process all data for a single tissue
    """
    tissue, deseq_file, gtf_file, output_dir = args
    
    # Process DESeq2 results
    gene_sets = process_deseq_results(deseq_file)
    
    # Prepare output directory
    tissue_output_dir = os.path.join(output_dir, tissue)
    os.makedirs(tissue_output_dir, exist_ok=True)
    
    # Create bed files
    bed_files = {}
    for category, genes in gene_sets.items():
        output_file = f"{tissue_output_dir}/{tissue}_{category}_regulated_genes.bed"
        bed_files[category] = output_file
        
        # Create the bed file directly (no parallelization here)
        create_bed_file((genes, gtf_file, output_file, category))
    
    return tissue, gene_sets, bed_files


In [5]:
# Configuration
deseq_files = {
    'Neuron': 'DEA_NEU.csv',
    'NSC': 'DEA_NSC.csv'
}
gtf_file = "gencode.vM10.annotation.gtf"  
output_dir = "metaprofile_results"
sample_sheet = "EXOGENOUS_sample_sheet.csv"
genome_size_file = "mm10.chrom.sizes"  

In [ ]:
# Create main output directory
os.makedirs(output_dir, exist_ok=True)

# Prepare arguments for parallel processing
process_args = [
    (tissue, deseq_file, gtf_file, output_dir)
    for tissue, deseq_file in deseq_files.items()
]

# Process tissues in parallel (single level of parallelization)
print(f"Processing {len(deseq_files)} tissues using {min(cpu_count(), len(deseq_files))} processes...")

with Pool(processes=min(cpu_count(), len(deseq_files))) as pool:
    results = pool.map(process_tissue_data, process_args)

# Organize results
tissue_gene_sets = {}
tissue_bed_files = {}

for tissue, gene_sets, bed_files in results:
    tissue_gene_sets[tissue] = gene_sets
    tissue_bed_files[tissue] = bed_files
    print(f"Completed processing {tissue}")

In [ ]:
print('Neuron')
for key in tissue_gene_sets['Neuron'].keys():
    print(key)
    print(len(tissue_gene_sets['Neuron'][key]))

print('\nNSC')
for key in tissue_gene_sets['NSC'].keys():
    print(key)
    print(len(tissue_gene_sets['NSC'][key]))


In [8]:
# Construct awk command to extract coordinates from GTF:
# -F'\t' : Set tab as field separator
# $3=="gene" : Only process lines where 3rd column is "gene"
# match($9, /gene_name "([^"]+)"/, n) : Extract gene name from 9th column attributes
# n[1]~/{gene_list}/ : Check if extracted gene name matches our gene list
# Print format: chromosome start end gene_name score strand
# Note: BED format is 0-based, so we subtract 1 from start coordinate ($4-1)

In [9]:
# def generate_metaprofiles(bed_files, sample_sheet, output_dir):
#     """
#     Generate metaprofiles using deepTools
#     """
#     # Read sample sheet
#     samples = pd.read_csv(sample_sheet)
    
#     for tissue in ['Neuron', 'NSC']:
#         tissue_samples = samples[samples['Tissue'] == tissue]
#         bam_files = tissue_samples['bamReads'].tolist()
        
#         for category, bed_file in bed_files.items():
#             # Generate matrix using computeMatrix
#             matrix_file = f"{output_dir}/{tissue}_{category}_matrix.gz"
#             profile_file = f"{output_dir}/{tissue}_{category}_profile.png"
            
#             cmd_matrix = f"""
#             computeMatrix scale-regions -S {' '.join(bam_files)} \
#                 -R {bed_file} \
#                 -b 5000 -a 5000 \
#                 --regionBodyLength 5000 \
#                 --skipZeros \
#                 -o {matrix_file}
#             """
            
#             cmd_plot = f"""
#             plotProfile -m {matrix_file} \
#                 --perGroup \
#                 --colors red blue \
#                 --plotTitle "{tissue} {category}-regulated genes" \
#                 -out {profile_file}
#             """
            
#             subprocess.run(cmd_matrix, shell=True)
#             subprocess.run(cmd_plot, shell=True)

In [14]:
def bam_to_bigwig(bam_file, output_dir, genome_size_file):
    """
    Convert BAM file to bigWig format using deeptools
    """
    basename = os.path.splitext(os.path.basename(bam_file))[0]
    output_bw = f"{output_dir}/{basename}.bw"
    
    if not os.path.exists(output_bw):
        print(f"Converting {basename} BAM to bigWig...")
        os.makedirs(output_dir, exist_ok=True)
        
        # Create unique temporary directory for this conversion
        with tempfile.TemporaryDirectory(prefix=f"tmp_{basename}_") as temp_dir:
            # Sort and index BAM
            print(f"- Sorting and indexing {basename} BAM...")
            sorted_bam = f"{temp_dir}/{basename}.sorted.bam"
            subprocess.run(f"samtools sort -T {temp_dir}/sort_tmp {bam_file} -o {sorted_bam}", 
                         shell=True, check=True)
            subprocess.run(f"samtools index {sorted_bam}", shell=True, check=True)
            
            # Convert directly to bigWig using bamCoverage
            print(f"- Converting {basename} to bigWig...")
            subprocess.run(f"""
                bamCoverage -b {sorted_bam} \
                    -o {output_bw} \
                    --binSize 10 \
                    --normalizeUsing RPKM \
                    --numberOfProcessors 1
            """, shell=True, check=True)
            
            print(f"Completed conversion of {basename} to bigWig")
    else:
        print(f"BigWig file already exists for {basename}, skipping conversion")
    
    return output_bw

def generate_metaprofiles(tissue_bed_files, sample_sheet, output_dir, genome_size_file):
    """
    Generate metaprofiles using deepTools with parallel processing
    """
    print("\nStarting metaprofile generation...")
    
    # Read sample sheet
    print("Reading sample sheet...")
    samples = pd.read_csv(sample_sheet)
    bigwig_dir = f"{output_dir}/bigwig"
    os.makedirs(bigwig_dir, exist_ok=True)
    
    # Process BAM files sequentially to avoid resource conflicts
    print("\nConverting BAM files to bigWig format...")
    bigwig_files = []
    for bam_file in samples['bamReads'].tolist():
        try:
            bigwig_file = bam_to_bigwig(bam_file, bigwig_dir, genome_size_file)
            bigwig_files.append(bigwig_file)
        except Exception as e:
            print(f"Error converting {bam_file}: {str(e)}")
            return
    
    samples['bigWig'] = bigwig_files
    
    # Create a list of all tasks to run
    print("\nPreparing metaprofile generation tasks...")
    tasks = []
    for tissue in ['Neuron', 'NSC']:
        tissue_samples = samples[samples['Tissue'] == tissue]
        tissue_bigwigs = tissue_samples['bigWig'].tolist()
        bed_files = tissue_bed_files[tissue]
        
        for category, bed_file in bed_files.items():
            # Create unique temporary directory for matrix files
            matrix_dir = f"{output_dir}/matrix_{tissue}_{category}"
            os.makedirs(matrix_dir, exist_ok=True)
            
            matrix_file = f"{matrix_dir}/{tissue}_{category}_matrix.gz"
            profile_file = f"{output_dir}/{tissue}_{category}_profile.png"
            
            cmd_matrix = f"""
            computeMatrix scale-regions -S {' '.join(tissue_bigwigs)} \
                -R {bed_file} \
                -b 5000 -a 5000 \
                --regionBodyLength 5000 \
                --skipZeros \
                --numberOfProcessors 1 \
                -o {matrix_file}
            """
            
            cmd_plot = f"""
            plotProfile -m {matrix_file} \
                --perGroup \
                --colors red blue \
                --plotTitle "{tissue} {category}-regulated genes" \
                -out {profile_file}
            """
            
            tasks.append((tissue, category, cmd_matrix, cmd_plot))

    # Process tasks sequentially to avoid resource conflicts
    print(f"\nExecuting {len(tasks)} metaprofile tasks...")
    for task in tasks:
        tissue, category, cmd_matrix, cmd_plot = task
        try:
            print(f"\nProcessing {tissue} {category}-regulated genes...")
            print("- Generating matrix...")
            subprocess.run(cmd_matrix, shell=True, check=True)
            print("- Creating profile plot...")
            subprocess.run(cmd_plot, shell=True, check=True)
            print(f"Completed processing {tissue} {category}-regulated genes")
        except subprocess.CalledProcessError as e:
            print(f"Error processing {tissue} {category}: {str(e)}")
            continue
    
    print("\nMetaprofile generation completed!")

In [ ]:
# Generate metaprofiles
generate_metaprofiles(tissue_bed_files, sample_sheet, output_dir, genome_size_file)

In [12]:
def plot_combined_profiles(output_dir):
    """
    Create combined visualization
    """
    try:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        categories = ['non', 'up', 'down']
        tissues = ['Neuron', 'NSC']
        
        for i, tissue in enumerate(tissues):
            for j, category in enumerate(categories):
                matrix_file = f"{output_dir}/{tissue}_{category}_matrix.gz"
                if os.path.exists(matrix_file):
                    data = np.loadtxt(matrix_file)
                    
                    x = np.linspace(-5000, 5000, data.shape[1])
                    mean_profile = data.mean(axis=0)
                    std_profile = data.std(axis=0)
                    
                    axes[i, j].plot(x, mean_profile)
                    axes[i, j].fill_between(x, 
                                          mean_profile - std_profile,
                                          mean_profile + std_profile,
                                          alpha=0.2)
                    
                    axes[i, j].set_title(f"{tissue} {category}-regulated")
                    axes[i, j].set_xlabel("Distance from TSS (bp)")
                    axes[i, j].set_ylabel("Average signal")
        
        plt.tight_layout()
        plt.savefig(f"{output_dir}/combined_profiles.png")
    except Exception as e:
        print(f"Error creating combined plot: {str(e)}")

In [ ]:
# Create combined visualization
plot_combined_profiles(output_dir)